<a href="https://colab.research.google.com/github/hmatny/question_answering/blob/master/Anserini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment Setup

#### Download and install maven

In [1]:
!wget http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
!tar -xvzf apache-maven-3.6.1-bin.tar.gz -C /opt

--2019-04-18 14:40:54--  http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
Resolving mirror.reverse.net (mirror.reverse.net)... 208.100.14.200
Connecting to mirror.reverse.net (mirror.reverse.net)|208.100.14.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9136463 (8.7M) [application/x-gzip]
Saving to: ‘apache-maven-3.6.1-bin.tar.gz’

apache-maven-3.6.1- 100%[===================>]   8.71M  10.7MB/s    in 0.8s    

2019-04-18 14:40:55 (10.7 MB/s) - ‘apache-maven-3.6.1-bin.tar.gz’ saved [9136463/9136463]

apache-maven-3.6.1/README.txt
apache-maven-3.6.1/LICENSE
apache-maven-3.6.1/NOTICE
apache-maven-3.6.1/lib/
apache-maven-3.6.1/lib/slf4j-api.license
apache-maven-3.6.1/lib/checker-compat-qual.license
apache-maven-3.6.1/lib/jsr250-api.license
apache-maven-3.6.1/lib/jcl-over-slf4j.license
apache-maven-3.6.1/lib/org.eclipse.sisu.plexus.license
apache-maven-3.6.1/lib/animal-sniffer-annotations.license
apache-maven-3.6.

#### Set environment variables

In [2]:
import os
os.environ['PATH'] += ':/usr/lib/jvm/java-8-openjdk-amd64/bin:/opt/apache-maven-3.6.1/bin'
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['M2_HOME'] = "/opt/apache-maven-3.6.0"

# sanity check
!mvn -version

Apache Maven 3.6.1 (d66c9c0b3152b2e69ee9bac180bb8fcc8e6af555; 2019-04-04T19:00:29Z)
Maven home: /opt/apache-maven-3.6.1
Java version: 1.8.0_191, vendor: Oracle Corporation, runtime: /usr/lib/jvm/java-8-openjdk-amd64/jre
Default locale: en_US, platform encoding: UTF-8
OS name: "linux", version: "4.14.79+", arch: "amd64", family: "unix"


#### Clone Anserini git repo

In [3]:
!git clone https://github.com/castorini/Anserini.git
!cd Anserini && mvn clean package appassembler:assemble

Cloning into 'Anserini'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 6928 (delta 21), reused 60 (delta 13), pack-reused 6828
Receiving objects: 100% (6928/6928), 11.13 MiB | 19.35 MiB/s, done.
Resolving deltas: 100% (3608/3608), done.
[INFO] Scanning for projects...
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.7.0/maven-compiler-plugin-3.7.0.pom (11 kB at 23 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/30/maven-plugins-30.pom (10 kB at 275 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/30/maven-parent-30.pom (41 kB at 961 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/18/apache-18.pom (16 kB at 423 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2

#### Get CAR17 corpus and index data

Leave this commented out! Don't run. This is for reference on how we indexed the large corpus. 

In [0]:
# !wget http://trec-car.cs.unh.edu/datareleases/v2.0/paragraphCorpus.v2.0.tar.xz
# !tar -xvf paragraphCorpus.v2.0.tar.xz -C /content/Anserini/

In [0]:
# !cd /content/Anserini/ && sh target/appassembler/bin/IndexCollection -collection CarCollection \
# -generator LuceneDocumentGenerator -threads 500000 -input /content/Anserini/paragraphCorpus -index \
# lucene-index.car17a.pos+docvectors+rawdocs -storePositions -storeDocvectors \
# -storeRawDocs

In [0]:
# tar index
# !cd /content/Anserini/ && tar czf lucene-index.car17-large.tar.gz lucene-index.car17a.pos+docvectors+rawdocs

In [0]:
# save index to gs bucket
# !gsutil cp /content/Anserini/lucene-index.car17-large.tar.gz gs://cs378_bert/lucene-index-large.tar.gz

Copying file:///content/Anserini/lucene-index.car17-large.tar.gz [Content-Type=application/x-tar]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/14.1 GiB.                                     


#### Download the Lucene index

In [14]:
!gsutil cp gs://cs378_bert/lucene-index-large.tar.gz /tmp/lucene-index-large.tar.gz
!cd /tmp && tar -xvzf lucene-index-large.tar.gz -C /content/Anserini/

lucene-index.car17a.pos+docvectors+rawdocs/
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.tip
lucene-index.car17a.pos+docvectors+rawdocs/_2.fnm
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.tim
lucene-index.car17a.pos+docvectors+rawdocs/_2.nvd
lucene-index.car17a.pos+docvectors+rawdocs/_2_Lucene50_0.tim
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.doc
lucene-index.car17a.pos+docvectors+rawdocs/_2_Lucene50_0.doc
lucene-index.car17a.pos+docvectors+rawdocs/_3.nvd
lucene-index.car17a.pos+docvectors+rawdocs/_0.fnm
lucene-index.car17a.pos+docvectors+rawdocs/_0_Lucene70_0.dvd
lucene-index.car17a.pos+docvectors+rawdocs/_1.tvx
lucene-index.car17a.pos+docvectors+rawdocs/_0.si
lucene-index.car17a.pos+docvectors+rawdocs/_1.fdx
lucene-index.car17a.pos+docvectors+rawdocs/_3.si
lucene-index.car17a.pos+docvectors+rawdocs/_2.nvm
lucene-index.car17a.pos+docvectors+rawdocs/_2.tvd
lucene-index.car17a.pos+docvectors+rawdocs/_0.fdx
lucene-index.car17a.pos+docvectors+rawdocs

#### Install dependencies

In [8]:
!pip3 install pyjnius

  Stored in directory: /root/.cache/pip/wheels/c1/2d/85/9884050da2f10b9f72b029f34bedef0993c339437aa956906f
Successfully built pyjnius


### Using Anserini 

In [0]:
# Only run this once
import jnius_config
jnius_config.set_classpath('Anserini/target/anserini-0.4.1-SNAPSHOT-fatjar.jar')

In [0]:
question = 'Why did Mark Twain call the late 19th century the gilded age?'

In [17]:
from jnius import autoclass
JString = autoclass('java.lang.String')
JSearcher = autoclass('io.anserini.search.SimpleSearcher')

searcher = JSearcher(JString('Anserini/lucene-index.car17a.pos+docvectors+rawdocs'))
hits = searcher.search(JString(question),20)

contexts = []
for ind, hit in enumerate(hits):
        print('hit #' + str(ind))
        print('docid: ' + str(hit.docid))
        print('score: ' + str(hit.score))
        print('content: ')
        print(hit.content)
        print()
        contexts.append(hit.content)
        
#send contexts to BERT....

hit #0
docid: de9eeda39d6aa3c31ebe4ec25573a66313cabdd1
score: 18.579999923706055
content: 
The Gilded Age in United States history is the late 19th century, from the 1870s to about 1900. The term for this period came into use in the 1920s and 1930s and was derived from writer Mark Twain's 1873 novel The Gilded Age: A Tale of Today, which satirized an era of serious social problems masked by a thin gold gilding. The early half of the Gilded Age roughly coincided with the middle portion of the Victorian era in Britain and the Belle Époque in France. It was preceded by the Reconstruction Era that ended in 1877 and was succeeded by the Progressive Era that began in the 1890s.

hit #1
docid: d8987fbf94097295bfae7f18725439b44a3d99bd
score: 18.523799896240234
content: 
The "Gilded Age" was a term that Mark Twain used to describe the period of the late 19th century when there had been a dramatic expansion of American wealth and prosperity. Reform of the Age included the Civil Service Act, whic